In [ ]:
TANGO_HOST = "10.164.10.193:10000"
visibility_ip = "10.165.21.2"
dish_ids = ["SKA001", "SKA036"]

In [ ]:
import json
import os
from datetime import datetime
from time import sleep

from tango import AttributeProxy, DeviceProxy

os.environ["TANGO_HOST"] = TANGO_HOST

csp_controller = "mid-csp/control/0"
csp_server = "mid-csp/subarray/01"
csp_controller_dp = DeviceProxy(csp_controller)
csp_dp = DeviceProxy(csp_server)

sleep(3)

with open(
    "../../data/mid_telescope/cbf/sys_params/initial_system_param.json", "r", encoding="utf-8"
) as json_data:
    d = json.load(json_data)
    csp_controller_dp.LoadDishCfg(json.dumps(d))

sleep(3)
print(csp_controller_dp.sourceDishVccConfig)
sleep(3)
csp_controller_dp.commandTimeout = 100
sleep(3)
csp_controller_dp.On([])


# WAIT 100 Seconds

In [ ]:
with open(
    "../../data/mid_telescope/csp/assign_resources.json", "r", encoding="utf-8"
) as json_data:
    assign_resources_csp = json.load(json_data)
    assign_resources_csp["dish"]["receptor_ids"] = dish_ids
    print(assign_resources_csp)
    csp_dp.AssignResources(json.dumps(assign_resources_csp))

sleep(3)

print(csp_dp.obsState)

In [ ]:
with open("../../data/mid_telescope/tmc/configure_scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_configure_scan = d["csp"]
    csp_configure_scan["cbf"]["fsp"][0]["output_host"] = [[]]
    csp_configure_scan["cbf"][
        "delay_model_subscription_point"
    ] = "ska_mid/tm_leaf_node/csp_subarray_01/delayModel"
    csp_configure_scan["cbf"]["fsp"][0]["output_host"][0] = [0, visibility_ip]
    csp_configure_scan["cbf"]["fsp"][0]["output_port"] = [[0, 21000, 1]]
    print(csp_configure_scan)
    csp_dp.configure(json.dumps(csp_configure_scan))

sleep(5)
print(csp_dp.obsState)

In [ ]:
print(csp_dp.obsState)

In [ ]:
from datetime import datetime, timedelta, timezone

from astropy.time import Time

SKA_EPOCH = "1999-12-31T23:59:28Z"
ska_epoch_utc = Time(SKA_EPOCH, scale="utc")
ska_epoch_tai = ska_epoch_utc.unix_tai

start_utc_time = Time(datetime.now(timezone.utc) + timedelta(seconds=5), scale="utc")

target_epoch = start_utc_time.unix_tai - ska_epoch_tai

with open("../../data/mid_telescope/tmc/delay_model.json", encoding="utf-8") as f:
    delay_model = json.load(f)
delayModelProxy = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray_01")
delay_model["start_validity_sec"] = target_epoch
print(json.dumps(delay_model))


tm_server = "ska_mid/tm_leaf_node/csp_subarray_01"
tm_dp = DeviceProxy(tm_server)
tm_dp.delayModel = json.dumps(delay_model)

sleep(5)

In [ ]:
with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(csp_scan)
    csp_dp.scan(json.dumps(csp_scan))

sleep(2)
print(csp_dp.obsState)

In [ ]:
csp_dp.EndScan()

In [ ]:
csp_dp.GoToIdle()
sleep(6)
csp_dp.ReleaseAllResources()

In [ ]:
csp_dp.ObsReset()

In [ ]:
csp_controller_dp.Off([])